<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Paquet-interpretation" data-toc-modified-id="Paquet-interpretation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Paquet interpretation</a></span><ul class="toc-item"><li><span><a href="#Decompose-the-paquet" data-toc-modified-id="Decompose-the-paquet-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Decompose the paquet</a></span></li><li><span><a href="#Mapping-the-Ids" data-toc-modified-id="Mapping-the-Ids-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Mapping the Ids</a></span><ul class="toc-item"><li><span><a href="#Gotta-need-to-decompile-the-source-code-first" data-toc-modified-id="Gotta-need-to-decompile-the-source-code-first-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Gotta need to decompile the source code first</a></span></li><li><span><a href="#Then-extract-the-mapping-of-the-Ids" data-toc-modified-id="Then-extract-the-mapping-of-the-Ids-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Then extract the mapping of the Ids</a></span></li></ul></li></ul></li><li><span><a href="#Deserialize" data-toc-modified-id="Deserialize-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Deserialize</a></span></li></ul></div>

# Paquet interpretation

## Decompose the paquet

In [71]:
class Packet:
    def __init__(self, packetPayLoad):
        self.payload = packetPayLoad
        self.header = self.get_header()
        self.id = self.get_id()
        self.size_of_size = self.get_size_of_size()
        self.size_packet = self.get_size_packet()
        self.size_message = self.get_size_message()
        
    def get_header(self):
        return ''.join(self.payload.split(':')[:2])
        
    def get_bin_header(self):
        scale = 16 #hexadecimal
        num_of_bits = 16
        header = self.header
        
        return bin(int(header, scale))[2:].zfill(num_of_bits)
        
    def get_id(self):
        if ~hasattr(self, "bin_header"):
            self.bin_header = self.get_bin_header()
        
        return int(self.bin_header[:14], 2)
    
    def get_size_of_size(self):
        if ~hasattr(self, "bin_header"):
            self.bin_header = self.get_bin_header()
        return int(self.bin_header[-2:], 2)
    
    def get_size_packet(self):
        return ''.join(self.payload.split(":")[2 : 2+self.size_of_size])
    
    def get_size_message(self):
        return int(self.size_packet, base=16)

In [46]:
my_packet = capture[1].tcp.payload
print(my_packet)

05:45:00:00:23:ea:10:00:03:01:7d:71:80:71:73:41:a6:c8:08:08:00:00:00


In [47]:
testing_a_packet_class = Packet(my_packet)

In [48]:
testing_a_packet_class.payload

'05:45:00:00:23:ea:10:00:03:01:7d:71:80:71:73:41:a6:c8:08:08:00:00:00'

In [49]:
print("The header hexadecimal value is %s" %(testing_a_packet_class.header))
print("The packet ID is %s while the size of the size is %s" %(testing_a_packet_class.id,
                                                             testing_a_packet_class.size_of_size))
print("The message size in hexa is then %s which represents %s bytes after conversion" 
      %(testing_a_packet_class.size_packet, testing_a_packet_class.size_message))

The header hexadecimal value is 0545
The packet ID is 337 while the size of the size is 1
The message size in hexa is then 00 which represents 0 bytes after conversion


## Load the Ids

In [95]:
from ids_mapping import load_ids

In [97]:
dict_ids = load_ids()

In [98]:
dict_ids

{'6089': 'ConsoleCommandsListMessage',
 '1163': 'AdminQuietCommandMessage',
 '232': 'AdminCommandMessage',
 '6197': 'ConsoleMessage',
 '6921': 'CredentialsAcknowledgementMessage',
 '261': 'IdentificationSuccessWithLoginTokenMessage',
 '5355': 'HelloConnectMessage',
 '5556': 'IdentificationAccountForceMessage',
 '4450': 'ServerSelectionMessage',
 '3': 'ServerStatusUpdateMessage',
 '5823': 'IdentificationFailedForBadVersionMessage',
 '9230': 'SelectedServerDataExtendedMessage',
 '9307': 'IdentificationMessage',
 '3593': 'IdentificationSuccessMessage',
 '1619': 'ServersListMessage',
 '1921': 'IdentificationFailedBannedMessage',
 '2277': 'SelectedServerRefusedMessage',
 '8715': 'SelectedServerDataMessage',
 '2694': 'IdentificationFailedMessage',
 '8658': 'MigratedServerListMessage',
 '1737': 'AcquaintanceSearchMessage',
 '2240': 'AcquaintanceServerListMessage',
 '4005': 'AcquaintanceSearchErrorMessage',
 '1022': 'AccountLinkRequiredMessage',
 '4407': 'NicknameRefusedMessage',
 '9581': 'Nic

**Searching for ID based on the previous packet**

In [63]:
id_packet = testing_a_packet_class.id

In [78]:
dictIds[str(id_packet)]

'GameMapMovementRequestMessage'

In [99]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

print(color.BOLD + 'Hello, World!' + color.END)

Hello, World!


In [100]:
for packet in list_packets_data:
    print("My packet is %s"%{packet})
    my_packet = Packet(packet)
    print(color.BOLD + dictIds[str(my_packet.id)] + color.END)
    print("------ NEXT PACKET ------")

My packet is {'42:f9:0d:a5:82:fc:d7:b1:16:94:b9:1f:e3:02:00:01'}
InteractiveUsedMessage
------ NEXT PACKET ------
My packet is {'05:45:00:00:23:ea:10:00:03:01:7d:71:80:71:73:41:a6:c8:08:08:00:00:00'}
GameMapMovementRequestMessage
------ NEXT PACKET ------
My packet is {'48:91:04:d5:01:d1:02:74:05:16:00:05:01:7d:01:7e:01:7f:01:80:01:73:00:04:42:1c:01:a0:04:94:00:00:93:9c'}
BasicAckMessage
------ NEXT PACKET ------
My packet is {'42:f9:0d:a5:82:e4:f0:ce:0c:94:b9:1f:e3:02:00:01'}
InteractiveUsedMessage
------ NEXT PACKET ------
My packet is {'69:5d:6f:0e:00:45:43:72:61:20:65:6e:20:67:61:6c:c3:a8:72:65:20:63:68:65:72:63:68:65:20:71:75:65:6c:71:75:27:75:6e:20:70:6f:75:72:20:70:61:73:73:65:72:20:63:68:6f:75:71:75:65:20:3a:29:20:54:70:20:70:6f:73:73:69:62:6c:65:20:21:63:ac:76:e3:00:08:77:36:33:33:34:65:37:69:42:66:76:8a:a0:24:a0:00:00:09:54:63:68:6f:75:6b:69:77:65:00:00:0a:2d:28:4c'}
ChatServerMessage
------ NEXT PACKET ------
My packet is {'74:05:18:00:06:00:c3:00:df:00:fb:01:17:01:33:01:4f:

ValueError: invalid literal for int() with base 16: ''

In [83]:
my_packet.size_packet

'7c'

In [85]:
debug_packet = Packet('01:a4:00:00:23:f2')

ValueError: invalid literal for int() with base 16: ''

In [94]:
"%s/%s" %('test', 'name')

'test/name'

# Deserialize